In [ ]:
import sys
sys.path.append('..')

# Maximum Likelihood Estimation (MLE)

## Learning objectives
- Understand mathematically, what it means to find "optimal" parameters
- Derive MLE from scratch
- Quantify the confidence in a particular parameterisation
- Understand why we use the mean squared error loss function for regression tasks rather than say the absolute or quartic squared error

## Intro - what distribution best describes our data?

![](images/mle_compare_dists.jpg)

## How can we quantify this?

The maximum likelihood parameters of a distribution are those which parameterise a given distribution such that the probability of sampling the observed data from it is maximised.

![](images/mle_deriv.jpg)

This objective, which results in a model that maximises the likelihood of observing our dataset, is a cross entropy loss.

## What is cross entropy?

Before we talk about more what it is, here's the mathematical defninition of the cross entropy between two distributions:

![](images/cross_entropy_definition.jpg)

Goodfellow et al clarify further that "any loss term that contains a negative log-likelihood is a cross entropy between the empirical distribution defined by the training set and the probability distribution defined by the model".
What this means should become more obvious over the next few paragraphs.

### The KL divergence

The aim of machine learning is to be able to model things that we care about. 
We want to be able to know the distribution of features (unsupervised problems) and labels (supervised problems) that appear in the data.
That is, we want to know the true data generating distribution.
Maximum likelihood **implicitly** minimises the difference between the empirical distribution $\hat{p}_{data}$ and the model distribution $p_{model}$.
This happens because what we minimise in a cross entropy loss, is the same term as that which appears in the equation for the **KL-divergence**, which is an expression which explicitly measures the difference between two distributions, once the terms which do not depend on our model parameterisation are removed.

![](images/kl_definition.jpg)

There is much, much more to understand about the KL-divergence... look into it.

### Minimising the cross entropy implicitly minimises the KL divergence

![](images/kl_cross_entropy_argmin.jpg)

Throughout the rest of the course, we will try to minimise the expression produced from the above derivation of the MLE parameters, which is a cross entropy between our empirical distribution and our model distribution.
This should make sense: our data that we want to model has some underlying distribution, and we want to build a model of the same distribution - so we will minimise their difference.
We'll do this by minimising the KL-divergence, which quantifies their difference.
But we don't actually have an expression for the empirical distribution which appears in the KL-divergence equation.
So instead of minimising the KL-divergence directly, we minimise the cross entropy, which has the same minimum point.

The KL-divergence and the cross entropy are related by the [Shannon entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory))

![](images/kl_cross_entropy_shannon_entropy_relationship.jpg)

## Let's visualise some unsupervised data (no labels) so that we can try to fit a distribution to it using maximum likelihood.

In [ ]:
# DON'T WORRY ABOUT THIS CELL, IT JUST SETS SOME STUFF UP
import numpy as np
import matplotlib.pyplot as plt
from utils import get_regression_data, show_regression_data

X, Y = get_regression_data()

X = X[:, 0]
# Y = np.zeros_like(Y)

plt.scatter(X, np.zeros(X.shape[0]), c='r')
plt.ylim(0, 0.03)
plt.show()

mu = np.mean(X)
sigma = np.std(X)
print('mean:', mu)
print('standard deviation:', sigma)


We will model our data with a Gaussian (normal distribution). Let's code one up now.

![](images/gaussian.jpg)

In [ ]:
import numpy as np

class GaussianPDF:
    def __init__(self, mu=0, sigma=1):
        self.mu = mu
        self.sigma = sigma

    def __call__(self, x):
        p_x = np.exp(-(x - self.mu)**2 / (2*self.sigma**2)) / ( np.sqrt(2*np.pi) * self.sigma) # pdf equation for gaussian
        return p_x

p = GaussianPDF(mu, sigma)

p(0)
domain = np.linspace(min(X)-1, max(X)+1)
plt.plot(domain, p(domain))
plt.scatter(X, np.zeros(X.shape[0]), c='r')
plt.ylim(0, 1)
plt.xlim(min(domain), max(domain))
plt.show()

What is the central position (mean) of the Gaussian distribution which best describes the feature values of the data, assuming that they are normally distributed?

Let's test out different Gaussian distributions by evaluating the likelihood of our dataset being sampled from each of them, using the formula we derived above.

In [ ]:

mus = np.linspace(min(X), max(X), 50)
objectives = np.zeros_like(mus)
best_mu = 0
best_val = -float('inf') # initialise best value as infinitely bad
for idx, m in enumerate(mus):
    p = GaussianPDF(m, sigma)
    obj = 0
    for x in X:
        obj += np.log(p(x)) # compute the log likelihood for this example and add it to the objective which we wish to maximise
    objectives[idx] = obj
    if obj > best_val:
        best_val = obj
        best_mu = m
    # print(obj)

print('BEST VAL:', best_val)
print('BEST MU:', best_mu)

plt.plot(mus, objectives)


## Now let's look at the supervised case

Let's now consider the case where our dataset contains labels as well as just example features.

We can follow the same steps to find an objective function for solving supervised regression problems. We simply model the conditional distribution over the labels given features, rather than modelling the distribution of features.

![](images/supervised_mle.jpg)

So for every input, we are going to have our model output a distribution, rather than a point estimate. 

We can do this by simply having our model output the parameters for a particular distribution over labels, such as the mean and variance of a Gaussian.

In this case we will assume that the variance of these distributions is fixed; it will not affect the argmax of the distribution anyway.

## Note the central limit theorem
Here, we assume that the distribution over labels **for a particular example** is Gaussian because of the [central limit theorem](https://www.youtube.com/watch?v=YAlJCEDH2uY).
The central limit theorem tells us that the mean of a sample of the same measurement will be normally distributed, even if the individual measurements themselves are not normally distributed.
So even if an example's observed label is noisy and is affected by random error, the distribution of the mean of labels assigned to it over many attempts will still be normally distributed.

For example, if a human labelling hand-written digits clicks an incorrect label with uniformly random probability, the mean of the labels which they assign to this example will be normally distributed about the true mean.

## Visualise labels as distributions
For now let's assume that we are modelling the labels of a particular example with a normal distribution.
Below we will take Gaussians of a fixed size, and plot them over the y-axis, centred with their mean at the position of the true label for now. 
These Gaussians vary in the positive x-direction, from the feature value, by the probability that this distribution assigns to any given label value.

As mentioned, the Gaussians are fixed in size by having a constant standard deviation and are scaled down in size so that they can be visualised easily.

In [ ]:
from get_colors import colors

def plot_dist(x, y, color='blue'):
    domain = np.linspace(min(Y), max(Y))
    p = GaussianPDF(y, 0.3)
    distributions = 0.05*p(domain) # scale down the size of the distributions for visualisation purposes
    p_plot = distributions + x # place the distribution extending from the x position of the example which it corresponds to
    plt.plot(p_plot, domain, c=colors[i%len(colors)]) # plot each distribution of predicted labels against
    plt.scatter(np.ones_like(domain)*x, domain, c=colors[i%len(colors)], marker='.', s=1)

for i in range(len(X)):
    x = X[i]
    y = Y[i]
    domain = np.linspace(min(Y), max(Y))
    p = GaussianPDF(y, 1)
    plt.scatter(x, y, c=colors[i%len(colors)])
    plot_dist(x, y)
plt.show()


## Now let's actually use our model to produce these distributions over labels for each input example

Our model will probably not produce the same Gaussians because it will likely have some error in predicting the label (mean of the Gaussian) for each example.


In [ ]:
grad = (Y[-1] - Y[0]) / (X[-1] - X[0])
print(grad)

class LinearModel:
    def __init__(self, w, b):
        self.w = w
        self.b = b

    def __call__(self, x):
        return self.b + self.w * x

model = LinearModel(grad, -8)

for i in range(len(X)):
    color = colors[i%len(colors)]
    if i == 0:
        plt.scatter(X[i], Y[i], c=color, label='ground truth')
        plt.scatter(X[i], model(X[i]), marker='x', c=color, label='prediction')
    plt.scatter(X[i], Y[i], c=color)
    plt.scatter(X[i], model(X[i]), marker='x')
    plot_dist(X[i], model(X[i]))
    plt.legend()
plt.plot(X, model(X))
plt.show()

## Linear regression as maximum likelihood

Previously we implemented a linear regression algorithm that made a point estimate of the label $y$ given an input $x$.
We chose to minimise the mean squared error somewhat arbitrarily. Why did we choose this as our criterion? Why not minimise the absolute error or the mean quartic error?

We want to find the parameters of the model that we have defined that best describe the data. That is, the maximum likelihood parameters $\theta_{ML}$.
Somehow we need to link the probability distributions produced by our model parameterisation.

Let's revisit the linear regression algorithm from a perspective of maximum likelihood.
Rather than predicting a point estimate, we now have the model predict a conditional probability distribution $p(y|x;\theta)$ over the label $y$ given $x$.
That means that our model is predicting the confidence of the label taking one of a range of possible values, rather than predicting the exact value, for each example input.

To yield the same linear regression learning algorithm, we assume that the distribution over labels for a given example is a Gaussian. Again, this is warranted by the central limit theorem.

## Deriving the mean squared error loss function from maximum likelihood

We know that maximum likelihood means maximising the probability of observing the data predicted by our model. Putting the equation for our assumedly Gaussian probability distribution into the objective that we defined earlier transforms it into the same objective that we minimised previously - the mean squared error.

![](images/linear_reg_as_mle.jpg)

This should highlight why the MSE loss is what we optimise for, rather than any other arbitrary loss function such as absolute error.

## Properties of maximum likelihood

### The maximum likelihood estimator is a **consistent** estimator
This means that as the number of training examples approaches infinity, the maximum likelihood estimate of the parameter converges to the true value of the parameter.

### Statistical efficiency


## Summary
- Maximum Likelihood Estimation is a procedure for finding parameters that best describe a set of data
- The reason that we minimise MSE is because it is the objective produced by plugging a Gaussian distribution into the maximum likelihood objective